# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.117760e+02     1.752143e+00
 * time: 0.08879995346069336
     1     1.127012e+01     9.010206e-01
 * time: 1.2960319519042969
     2    -1.168907e+01     1.035015e+00
 * time: 1.4277348518371582
     3    -3.391287e+01     7.937160e-01
 * time: 1.5887279510498047
     4    -4.748170e+01     5.294693e-01
 * time: 1.7372148036956787
     5    -5.687262e+01     2.031639e-01
 * time: 1.8709847927093506
     6    -5.982740e+01     1.362444e-01
 * time: 1.9846489429473877
     7    -6.098900e+01     5.330095e-02
 * time: 2.1105737686157227
     8    -6.144817e+01     4.543621e-02
 * time: 2.2239789962768555
     9    -6.167400e+01     4.344837e-02
 * time: 2.3315367698669434
    10    -6.185840e+01     2.655818e-02
 * time: 2.444506883621216
    11    -6.198231e+01     2.265142e-02
 * time: 2.551302909851074
    12    -6.204929e+01     1.726648e-02
 * time: 2.6648008823394775
    13    -6.209322e+01     1.383604e-02
 * time: 2.76941680908

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671070
    AtomicLocal         -18.8557647
    AtomicNonlocal      14.8522616
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485368 
    Xc                  -19.3336819

    total               -62.261666461254
